# Import Libraries

In [2]:
from pinecone.grpc import PineconeGRPC as PineconeGRPC_Client
from langchain_pinecone import Pinecone as LangChainPinecone


from pinecone import ServerlessSpec

from langchain_openai import OpenAI, OpenAIEmbeddings


from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv
load_dotenv()
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter


d:\DigiMark\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\.venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Loading Documents

In [3]:
file_path = r'D:\DigiMark\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\monkey_story.txt'

In [4]:
loader = TextLoader(file_path) 

In [5]:
documents = loader.load()

In [6]:
documents[0].metadata

{'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\monkey_story.txt'}

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
chunks = text_splitter.split_documents(documents)

In [43]:
help(RecursiveCharacterTextSplitter)

Help on class RecursiveCharacterTextSplitter in module langchain_text_splitters.character:

class RecursiveCharacterTextSplitter(langchain_text_splitters.base.TextSplitter)
 |  RecursiveCharacterTextSplitter(separators: 'Optional[List[str]]' = None, keep_separator: "Union[bool, Literal['start', 'end']]" = True, is_separator_regex: 'bool' = False, **kwargs: 'Any') -> 'None'
 |  
 |  Splitting text by recursively look at characters.
 |  
 |  Recursively tries to split by different characters to find one
 |  that works.
 |  
 |  Method resolution order:
 |      RecursiveCharacterTextSplitter
 |      langchain_text_splitters.base.TextSplitter
 |      langchain_core.documents.transformers.BaseDocumentTransformer
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, separators: 'Optional[List[str]]' = None, keep_separator: "Union[bool, Literal['start', 'end']]" = True, is_separator_regex: 'bool' = False, **kwargs: 'Any') -> 'None'
 |      Create a new

In [8]:
chunks[0]

Document(metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\monkey_story.txt'}, page_content='In the lush canopy of a tropical rainforest, two mischievous monkeys, Coco and Mango, swung from branch to branch, their playful antics echoing through the trees. \n\n\n\nThey were inseparable companions, sharing everything from juicy fruits to secret hideouts high above the forest floor. One day, while exploring a new part of the forest, Coco stumbled upon a beautiful orchid hidden among the foliage.')

# CharacterTextSplitter

In [29]:
text_splitter = CharacterTextSplitter(
    separator=r"\n+",  # Regex to match one or more newlines
    chunk_size=1000,    # Adjust chunk size as needed
    chunk_overlap=200,    # Optional overlap
    is_separator_regex=True
)

In [99]:
chunks=text_splitter.split_documents(documents=documents)

Created a chunk of size 1342, which is longer than the specified 1000


In [100]:
for indx in chunks:
    print("-",indx.page_content)

- In the lush canopy of a tropical rainforest, two mischievous monkeys, Coco and Mango, swung from branch to branch, their playful antics echoing through the trees. They were inseparable companions, sharing everything from juicy fruits to secret hideouts high above the forest floor. One day, while exploring a new part of the forest, Coco stumbled upon a beautiful orchid hidden among the foliage. \n+Entranced by its delicate petals, Coco plucked it and presented it to Mango with a wide grin. Overwhelmed by Coco's gesture of friendship, Mango hugged Coco tightly, cherishing the bond they shared. From that day on, Coco and Mango ventured through the forest together, their friendship growing stronger with each passing adventure. \n+As they watched the sun dip below the horizon, casting a golden glow over the treetops, they knew that no matter what challenges lay ahead, they would always have each other, and their hearts brimmed with joy.
- Crocodiles (family Crocodylidae) or true crocodile

In [96]:
chunks

[Document(metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\monkey_story.txt'}, page_content="In the lush canopy of a tropical rainforest, two mischievous monkeys, Coco and Mango, swung from branch to branch, their playful antics echoing through the trees. They were inseparable companions, sharing everything from juicy fruits to secret hideouts high above the forest floor. One day, while exploring a new part of the forest, Coco stumbled upon a beautiful orchid hidden among the foliage. \\n+Entranced by its delicate petals, Coco plucked it and presented it to Mango with a wide grin. Overwhelmed by Coco's gesture of friendship, Mango hugged Coco tightly, cherishing the bond they shared. From that day on, Coco and Mango ventured through the forest together, their friendship growing stronger with each passing adventure. \\n+As they watched the sun dip below the horizon, casting a golden glow over the treetops, they knew that no matte

In [20]:
from langchain_community.document_loaders import PyPDFLoader

In [21]:
loader=PyPDFLoader(r"D:\DigiMark\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\data\case_law 2\0009-01.pdf")

In [22]:
pdf=loader.load()

In [23]:
pdf

[Document(metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\data\\case_law 2\\0009-01.pdf', 'page': 0}, page_content='REPORTS\nOP\nCASES\nARGUED\nAND\nDETERMINED\nTERM,\nJANUARY\n1840.\nMontgomery\nBranch\nBank\nat\nv.\nHarrison.\ngiven\ncarry\n1.\nat\npayment\npurchase\nNotes\ninterest\nin\nSections\nfor\nthe\nof\nSixteenth\ncent,\nannum,\nper\nj\nudgment\non\nper\npaid,\nthe\nrate\nof\nis\nobtained\nsix\nuntil\nuntil\nor\nthem,\nchanged.\nor\nthe\ncontract\notherwise\nError\nCourt\nof\nCounty.\nto\nthe\nCircuit\nMontgomery\nthe\ndefendant\nTHIS\nthe\ncourt\nbelow\nby\naction\nwas\nin\nbrought\nin\nof\nto\nhave\nerror,\na\nsum\nto\nof\nthe\nBank\nmoney\nalleged\nrecover\nnote\nbeen\nthe\nBank\non\na\nand\nreceived\nby\ndemanded\nimproperly\nThe\nsection.\nsixteenth\nfor\nof\na\nof\na\ngiven\nthe\npari\npurchase\nnote\nreads\nthus:\n“\nto\nOn\nwe\n1838,\npromise\nor\nof\nJanuary,\nbefore\nthe\n1st\nday\nof\nthe\nBank\nto\nof\nthe

In [14]:
import re

In [ ]:
def clean_pdf_text(text: str) -> str:
    # Remove unwanted newlines and extra spaces
    text = text.replace("\n", " ")  # Replace all newlines with a space
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    return text.strip()  # Remove any leading or trailing whitespace

# Example usage
cleaned_text = clean_pdf_text(pdf[0].page_content)

In [116]:
cleaned_text

'REPORTS OP CASES ARGUED AND DETERMINED TERM, JANUARY 1840. Montgomery Branch Bank at v. Harrison. given carry 1. at payment purchase Notes interest in Sections for the of Sixteenth cent, annum, per j udgment on per paid, the rate of is obtained six until until or them, changed. or the contract otherwise Error Court of County. to the Circuit Montgomery the defendant THIS the court below by action was in brought in of to have error, a sum to of the Bank money alleged recover note been the Bank on a and received by demanded improperly The section. sixteenth for of a of a given the pari purchase note reads thus: “ to On we 1838, promise or of January, before the 1st day of the Bank to of the of the Branch pay President and Directors four of the the twenty State of sum Alabama at Montgomery, cents, a half paya- hundred and twelve and dollars seven seventy received, part being ble and at said for Bank, value negotiable'

In [112]:
pdf[0].page_content

'REPORTS\nOP\nCASES\nARGUED\nAND\nDETERMINED\nTERM,\nJANUARY\n1840.\nMontgomery\nBranch\nBank\nat\nv.\nHarrison.\ngiven\ncarry\n1.\nat\npayment\npurchase\nNotes\ninterest\nin\nSections\nfor\nthe\nof\nSixteenth\ncent,\nannum,\nper\nj\nudgment\non\nper\npaid,\nthe\nrate\nof\nis\nobtained\nsix\nuntil\nuntil\nor\nthem,\nchanged.\nor\nthe\ncontract\notherwise\nError\nCourt\nof\nCounty.\nto\nthe\nCircuit\nMontgomery\nthe\ndefendant\nTHIS\nthe\ncourt\nbelow\nby\naction\nwas\nin\nbrought\nin\nof\nto\nhave\nerror,\na\nsum\nto\nof\nthe\nBank\nmoney\nalleged\nrecover\nnote\nbeen\nthe\nBank\non\na\nand\nreceived\nby\ndemanded\nimproperly\nThe\nsection.\nsixteenth\nfor\nof\na\nof\na\ngiven\nthe\npari\npurchase\nnote\nreads\nthus:\n“\nto\nOn\nwe\n1838,\npromise\nor\nof\nJanuary,\nbefore\nthe\n1st\nday\nof\nthe\nBank\nto\nof\nthe\nof\nthe\nBranch\npay\nPresident\nand\nDirectors\nfour\nof\nthe\nthe\ntwenty\nState\nof\nsum\nAlabama\nat\nMontgomery,\ncents,\na\nhalf\npaya-\nhundred\nand\ntwelve\nand\ndo

In [128]:
pdf[0].page_content

'REPORTS\nOP\nCASES\nARGUED\nAND\nDETERMINED\nTERM,\nJANUARY\n1840.\nMontgomery\nBranch\nBank\nat\nv.\nHarrison.\ngiven\ncarry\n1.\nat\npayment\npurchase\nNotes\ninterest\nin\nSections\nfor\nthe\nof\nSixteenth\ncent,\nannum,\nper\nj\nudgment\non\nper\npaid,\nthe\nrate\nof\nis\nobtained\nsix\nuntil\nuntil\nor\nthem,\nchanged.\nor\nthe\ncontract\notherwise\nError\nCourt\nof\nCounty.\nto\nthe\nCircuit\nMontgomery\nthe\ndefendant\nTHIS\nthe\ncourt\nbelow\nby\naction\nwas\nin\nbrought\nin\nof\nto\nhave\nerror,\na\nsum\nto\nof\nthe\nBank\nmoney\nalleged\nrecover\nnote\nbeen\nthe\nBank\non\na\nand\nreceived\nby\ndemanded\nimproperly\nThe\nsection.\nsixteenth\nfor\nof\na\nof\na\ngiven\nthe\npari\npurchase\nnote\nreads\nthus:\n“\nto\nOn\nwe\n1838,\npromise\nor\nof\nJanuary,\nbefore\nthe\n1st\nday\nof\nthe\nBank\nto\nof\nthe\nof\nthe\nBranch\npay\nPresident\nand\nDirectors\nfour\nof\nthe\nthe\ntwenty\nState\nof\nsum\nAlabama\nat\nMontgomery,\ncents,\na\nhalf\npaya-\nhundred\nand\ntwelve\nand\ndo

In [ ]:
def clean_pdf_text(pdf):

    text=""
    for doc in pdf:
        # print(doc.page_content)
        text = doc.page_content.replace("\n", " ")  # Replace all newlines with a space
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
        print(text)
    # return text.strip()  # Remove any leading or trailing whitespace

# Example usage
cleaned_text = clean_pdf_text(pdf)

REPORTS OP CASES ARGUED AND DETERMINED TERM, JANUARY 1840. Montgomery Branch Bank at v. Harrison. given carry 1. at payment purchase Notes interest in Sections for the of Sixteenth cent, annum, per j udgment on per paid, the rate of is obtained six until until or them, changed. or the contract otherwise Error Court of County. to the Circuit Montgomery the defendant THIS the court below by action was in brought in of to have error, a sum to of the Bank money alleged recover note been the Bank on a and received by demanded improperly The section. sixteenth for of a of a given the pari purchase note reads thus: “ to On we 1838, promise or of January, before the 1st day of the Bank to of the of the Branch pay President and Directors four of the the twenty State of sum Alabama at Montgomery, cents, a half paya- hundred and twelve and dollars seven seventy received, part being ble and at said for Bank, value negotiable 
ALABAMA. 10 Montgomery Harrison.- v. Bank at Branch 16, 12, R. and See. 

In [122]:
for indx in pdf:
    print("-",indx)

- page_content='REPORTS
OP
CASES
ARGUED
AND
DETERMINED
TERM,
JANUARY
1840.
Montgomery
Branch
Bank
at
v.
Harrison.
given
carry
1.
at
payment
purchase
Notes
interest
in
Sections
for
the
of
Sixteenth
cent,
annum,
per
j
udgment
on
per
paid,
the
rate
of
is
obtained
six
until
until
or
them,
changed.
or
the
contract
otherwise
Error
Court
of
County.
to
the
Circuit
Montgomery
the
defendant
THIS
the
court
below
by
action
was
in
brought
in
of
to
have
error,
a
sum
to
of
the
Bank
money
alleged
recover
note
been
the
Bank
on
a
and
received
by
demanded
improperly
The
section.
sixteenth
for
of
a
of
a
given
the
pari
purchase
note
reads
thus:
“
to
On
we
1838,
promise
or
of
January,
before
the
1st
day
of
the
Bank
to
of
the
of
the
Branch
pay
President
and
Directors
four
of
the
the
twenty
State
of
sum
Alabama
at
Montgomery,
cents,
a
half
paya-
hundred
and
twelve
and
dollars
seven
seventy
received,
part
being
ble
and
at
said
for
Bank,
value
negotiable
' metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-D

In [17]:
def clean_pdf_text(pdf):

    text=""
    for doc in pdf:
        text = doc.page_content.replace("\n", " ")  # Replace all newlines with a space
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
        print(text)
        doc.page_content=text
    return pdf
# Example usage
cleaned_pdf = clean_pdf_text(pdf)

REPORTS OP CASES ARGUED AND DETERMINED TERM, JANUARY 1840. Montgomery Branch Bank at v. Harrison. given carry 1. at payment purchase Notes interest in Sections for the of Sixteenth cent, annum, per j udgment on per paid, the rate of is obtained six until until or them, changed. or the contract otherwise Error Court of County. to the Circuit Montgomery the defendant THIS the court below by action was in brought in of to have error, a sum to of the Bank money alleged recover note been the Bank on a and received by demanded improperly The section. sixteenth for of a of a given the pari purchase note reads thus: “ to On we 1838, promise or of January, before the 1st day of the Bank to of the of the Branch pay President and Directors four of the the twenty State of sum Alabama at Montgomery, cents, a half paya- hundred and twelve and dollars seven seventy received, part being ble and at said for Bank, value negotiable 
ALABAMA. 10 Montgomery Harrison.- v. Bank at Branch 16, 12, R. and See. 

In [25]:
cleaned_pdf

[Document(metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\data\\case_law 2\\0009-01.pdf', 'page': 0}, page_content='REPORTS OP CASES ARGUED AND DETERMINED TERM, JANUARY 1840. Montgomery Branch Bank at v. Harrison. given carry 1. at payment purchase Notes interest in Sections for the of Sixteenth cent, annum, per j udgment on per paid, the rate of is obtained six until until or them, changed. or the contract otherwise Error Court of County. to the Circuit Montgomery the defendant THIS the court below by action was in brought in of to have error, a sum to of the Bank money alleged recover note been the Bank on a and received by demanded improperly The section. sixteenth for of a of a given the pari purchase note reads thus: “ to On we 1838, promise or of January, before the 1st day of the Bank to of the of the Branch pay President and Directors four of the the twenty State of sum Alabama at Montgomery, cents, a half paya- hundred

In [27]:
clean_pdf_text(pdf)

REPORTS OP CASES ARGUED AND DETERMINED TERM, JANUARY 1840. Montgomery Branch Bank at v. Harrison. given carry 1. at payment purchase Notes interest in Sections for the of Sixteenth cent, annum, per j udgment on per paid, the rate of is obtained six until until or them, changed. or the contract otherwise Error Court of County. to the Circuit Montgomery the defendant THIS the court below by action was in brought in of to have error, a sum to of the Bank money alleged recover note been the Bank on a and received by demanded improperly The section. sixteenth for of a of a given the pari purchase note reads thus: “ to On we 1838, promise or of January, before the 1st day of the Bank to of the of the Branch pay President and Directors four of the the twenty State of sum Alabama at Montgomery, cents, a half paya- hundred and twelve and dollars seven seventy received, part being ble and at said for Bank, value negotiable 
ALABAMA. 10 Montgomery Harrison.- v. Bank at Branch 16, 12, R. and See. 

[Document(metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\data\\case_law 2\\0009-01.pdf', 'page': 0}, page_content='REPORTS OP CASES ARGUED AND DETERMINED TERM, JANUARY 1840. Montgomery Branch Bank at v. Harrison. given carry 1. at payment purchase Notes interest in Sections for the of Sixteenth cent, annum, per j udgment on per paid, the rate of is obtained six until until or them, changed. or the contract otherwise Error Court of County. to the Circuit Montgomery the defendant THIS the court below by action was in brought in of to have error, a sum to of the Bank money alleged recover note been the Bank on a and received by demanded improperly The section. sixteenth for of a of a given the pari purchase note reads thus: “ to On we 1838, promise or of January, before the 1st day of the Bank to of the of the Branch pay President and Directors four of the the twenty State of sum Alabama at Montgomery, cents, a half paya- hundred

In [28]:
pdf

[Document(metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\data\\case_law 2\\0009-01.pdf', 'page': 0}, page_content='REPORTS OP CASES ARGUED AND DETERMINED TERM, JANUARY 1840. Montgomery Branch Bank at v. Harrison. given carry 1. at payment purchase Notes interest in Sections for the of Sixteenth cent, annum, per j udgment on per paid, the rate of is obtained six until until or them, changed. or the contract otherwise Error Court of County. to the Circuit Montgomery the defendant THIS the court below by action was in brought in of to have error, a sum to of the Bank money alleged recover note been the Bank on a and received by demanded improperly The section. sixteenth for of a of a given the pari purchase note reads thus: “ to On we 1838, promise or of January, before the 1st day of the Bank to of the of the Branch pay President and Directors four of the the twenty State of sum Alabama at Montgomery, cents, a half paya- hundred

In [39]:
text_splitter = CharacterTextSplitter(
    # separator=r"\n+",  # Regex to match one or more newlines
    chunk_size=100,    # Adjust chunk size as needed
    chunk_overlap=50,    # Optional overlap
    is_separator_regex=True
)

In [40]:
chunks=text_splitter.split_documents(documents=pdf)

In [41]:
chunks

[Document(metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\data\\case_law 2\\0009-01.pdf', 'page': 0}, page_content='REPORTS OP CASES ARGUED AND DETERMINED TERM, JANUARY 1840. Montgomery Branch Bank at v. Harrison. given carry 1. at payment purchase Notes interest in Sections for the of Sixteenth cent, annum, per j udgment on per paid, the rate of is obtained six until until or them, changed. or the contract otherwise Error Court of County. to the Circuit Montgomery the defendant THIS the court below by action was in brought in of to have error, a sum to of the Bank money alleged recover note been the Bank on a and received by demanded improperly The section. sixteenth for of a of a given the pari purchase note reads thus: “ to On we 1838, promise or of January, before the 1st day of the Bank to of the of the Branch pay President and Directors four of the the twenty State of sum Alabama at Montgomery, cents, a half paya- hundred

In [42]:
pdf[0]

Document(metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\data\\case_law 2\\0009-01.pdf', 'page': 0}, page_content='REPORTS OP CASES ARGUED AND DETERMINED TERM, JANUARY 1840. Montgomery Branch Bank at v. Harrison. given carry 1. at payment purchase Notes interest in Sections for the of Sixteenth cent, annum, per j udgment on per paid, the rate of is obtained six until until or them, changed. or the contract otherwise Error Court of County. to the Circuit Montgomery the defendant THIS the court below by action was in brought in of to have error, a sum to of the Bank money alleged recover note been the Bank on a and received by demanded improperly The section. sixteenth for of a of a given the pari purchase note reads thus: “ to On we 1838, promise or of January, before the 1st day of the Bank to of the of the Branch pay President and Directors four of the the twenty State of sum Alabama at Montgomery, cents, a half paya- hundred 

In [ ]:
[Document(metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\data\\case_law 2\\0009-01.pdf', 'page': 0}, page_content='REPORTS OP  '),
 Document(metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\data\\case_law 2\\0009-01.pdf', 'page': 1}, page_content="ALABAMA. 10 Montgomery Harrison.- v.ate "),
 Document(metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\data\\case_law 2\\0009-01.pdf', 'page': 2}, page_content='TERM, JANUA